ref : https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student

In [1]:
import pandas as pd
import numpy as np
# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
pd.set_option('display.max_row',None)
pd.set_option('display.max_column',None)

In [2]:
path = '/mnt/d/SKH/DevStudy/JupyterNotebook/BITAmin12/시계열_1조/data/'

In [3]:
articles_df = pd.read_csv(path+'scraping/articles_df.csv',index_col=0)
print(articles_df.shape)
articles_df.head(2)

(17653, 4)


,title,date,url,text
0,Netflix and the 'stolen' indies,2018-01-01,https://www.polygon.com/2018/1/1/16829714/netf...,Netflix plans to release 80 original films in ...
1,‘The End Of The F***ing World’: Netflix Sets P...,2018-01-01,https://deadline.com/2018/01/the-end-of-the-fi...,Netflix has released the official trailer and ...


In [56]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
# model = AutoModelForSequenceClassification.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")

In [5]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-classification",
#                 model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
#                return_all_scores=True)

In [4]:
import re
import nltk
# nltk.download('all', download_dir='./nltk_data/')
nltk.data.path.append('./nltk_data/')

def preprocess(row):
    text = row['title']+' '+row['text'] # text에 제목 추가
    text = text.replace('\\n',' ') # 줄바꿈 문자열 제거
    text = re.sub("[^a-zA-Z|.,?!'$]", " ", text) # 영어와 의미있는 특수문자만 남기기
    text = re.sub("\s\s\s\s\s\s\s", " ", text) # 쓸데없이 큰 띄어쓰기 줄이기
    text = re.sub("\s\s\s\s\s\s", " ", text) # 쓸데없이 큰 띄어쓰기 줄이기
    text = re.sub("\s\s\s\s\s", " ", text) # 쓸데없이 큰 띄어쓰기 줄이기
    text = re.sub("\s\s\s\s", " ", text) # 쓸데없이 큰 띄어쓰기 줄이기
    text = re.sub("\s\s\s", " ", text) # 쓸데없이 큰 띄어쓰기 줄이기
    text = re.sub("\s\s", " ", text) # 쓸데없이 큰 띄어쓰기 줄이기
    text = text.replace('netflix','')
    # stopwords = nltk.corpus.stopwords.words('english')
    text = text.lower()
    
    row['text'] = text
    return row

In [5]:
articles_df_processed = articles_df.apply(preprocess,axis=1)

In [6]:
from transformers import pipeline

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    top_k=None
)

In [7]:
# english
distilled_student_sentiment_classifier ("I love this movie and i would watch it again and again!")
# >> [[{'label': 'positive', 'score': 0.9731044769287109},
#   {'label': 'neutral', 'score': 0.016910076141357422},
#   {'label': 'negative', 'score': 0.009985478594899178}]]

[[{'label': 'positive', 'score': 0.9731044769287109},
  {'label': 'neutral', 'score': 0.016910076141357422},
  {'label': 'negative', 'score': 0.009985478594899178}]]

In [8]:
# 문장 토큰화 진행
from nltk.tokenize import sent_tokenize

articles_df_processed['text_tokenized'] = articles_df_processed['text'].apply(sent_tokenize)
articles_df_processed['sent_length'] = articles_df_processed['text_tokenized'].apply(len)

def remove_stopwords(row):
    sentence_tokens = row['text_tokenized']
    stopwords = nltk.corpus.stopwords.words('english')
    filtered_tokens = []
    for st in sentence_tokens:
        for word in st.split(' '):
            if word not in stopwords:
                filtered_tokens.append(word)
    row['text_tokenized']=filtered_tokens
    return row

articles_df_processed = articles_df_processed.apply(remove_stopwords,axis=1)

In [11]:
### ⚠️ 문장 토큰화를 해서 sequence lenth가 512 이상인 row는 제외
# 날짜 골고루 있으니까 그냥 drop해도 될 듯
b=articles_df_processed.loc[articles_df_processed.sent_length>512,'date']
b

3184     2018-12-11
3343     2018-12-19
6849     2019-12-16
9974     2020-10-20
13951    2021-12-15
14172    2021-12-27
19933    2023-07-28
Name: date, dtype: object

In [10]:
drop_idx = articles_df_processed.loc[articles_df_processed.sent_length>512].index
articles_df_processed2 = articles_df_processed.drop(index=drop_idx)
print(articles_df_processed2.shape)
print(articles_df_processed2['sent_lenght'].max()) # max sequence lenght

(17646, 6)


KeyError: 'sent_lenght'

In [82]:
from nltk.tokenize import sent_tokenize
test_sent = sent_tokenize(articles_df_processed2.loc[0,'text'])
test_result = distilled_student_sentiment_classifier(test_sent)
print(test_result[0:2])

articles_df_processed2[['neg_score','neu_score','pos_score']]=0,0,0

def sum_sentiment_clf(result):
    negative_score = 0
    neutral_score = 0
    positive_score = 0
    for sentence_result in result:
        negative_score += sentence_result[0]['score']
        neutral_score += sentence_result[1]['score']
        positive_score += sentence_result[2]['score']
    return negative_score, neutral_score, positive_score

def sentiment_clf_by_sentences(row):
    text_tokenized = row['text_tokenized']
    result = distilled_student_sentiment_classifier(text_tokenized)
    print(f'{row["date"]} clear')
    negative_score, neutral_score, positive_score = sum_sentiment_clf(result)
    row['neg_score'] = negative_score
    row['neu_score'] = neutral_score
    row['pos_score'] = positive_score
    return row

[[{'label': 'negative', 'score': 0.5441957116127014}, {'label': 'neutral', 'score': 0.2291233092546463}, {'label': 'positive', 'score': 0.2266809642314911}], [{'label': 'positive', 'score': 0.5727243423461914}, {'label': 'negative', 'score': 0.2813911437988281}, {'label': 'neutral', 'score': 0.14588452875614166}]]


In [83]:
articles_df_processed2 = articles_df_processed2.apply(sentiment_clf_by_sentences,axis=1)

2018-01-01 clear
2018-01-01 clear


RuntimeError: The size of tensor a (579) must match the size of tensor b (512) at non-singleton dimension 1

In [84]:
test = articles_df_processed2.iloc[2]
distilled_student_sentiment_classifier(test['text_tokenized'])

RuntimeError: The size of tensor a (579) must match the size of tensor b (512) at non-singleton dimension 1

In [87]:
tokens = test['text_tokenized']
for t in tokens:
    print(len(t))

161
173
130
130
130
15
153
87
109
102
73
483
222
1176
670
1261
2533
926
17
582
999
